In [ ]:
import os
os.chdir('..')
import torch
import torch.nn as nn
from pathlib import Path
from src.data import prepare_data

torch.manual_seed(42)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear_layer = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, _ = self.rnn(x)
        out = self.linear_layer(output[:, -1])
        return out

In [ ]:
X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, scaler = prepare_data()

model = LSTMModel(1, 64, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

NUM_EPOCH = 50

best_val_loss = float("inf")

for epoch in range(NUM_EPOCH):
    print(f"== Epoch {epoch + 1}/{NUM_EPOCH} ==")
    optimizer.zero_grad()
    predictions = model(X_train_seq).squeeze()
    loss = criterion(predictions, y_train_seq)
    print(f"Loss: {loss.item()}")
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = criterion(model(X_val_seq).squeeze(), y_val_seq)
    print(f"Val Loss: {val_loss.item()}")
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        os.makedirs("models", exist_ok=True)
        torch.save(model.state_dict(), "models/best_model.pth")
        print(f"Best model saved!\nVal Loss: {best_val_loss} in epoch {epoch + 1}")
    model.train()